In [1]:
import os
import pandas as pd

import sqlite3
import logging
from datetime import datetime

In [2]:
path_to_files = r'/mnt/disks/disk-1/d_light/'
db_path = r'/mnt/disks/disk-1/d_light/sfc.db'


In [3]:
filename=r'./log/{}.log'.format(str(datetime.now().strftime("%Y%m%d%H%M%S")))
fmt = '%(asctime)s %(module)s %(levelname)s: %(message)s'
logging.basicConfig(filename=filename, filemode='w', level='INFO', format=fmt, datefmt ="%Y-%m-%d %H:%M:%S")


In [4]:
for root, dirs, files in os.walk(path_to_files):
    for name in files:
        logging.debug(name)
        if 'Payments' in name:
            logging.info(name)
            try:
                conn = sqlite3.connect(db_path)
                df = pd.read_csv(os.path.join(root, name),
                                dtype={'TransactionTs':'object',
                                        'Source':'str',
                                        'ContractId':'Int64',
                                        'CustomerId':'Int64',
                                        'PaymentResultTypeEntity':'str',
                                        'paymentStatusTypeEntity':'str',
                                        'AmountPaid':'Int64',
                                        'NumberOfTokens':'Int64',
                                        'Duration':'float64',
                                        },
                                       ) 
                df = df[df['paymentStatusTypeEntity'] != 'REFUSED']  ##
                df= df[df['PaymentResultTypeEntity'] != 'PAYMENT_FREE_CREDIT']  ##

                df.to_sql('Payments', con=conn, if_exists='append', index=False)
                conn.commit()
                conn.close()
                logging.info('{} committed successfully'.format(name))
                del df
            except Exception as e:
                logging.exception('read_csv failed: {} \n {}'.format(name, e))

        if 'Contracts' in name:
            pass # only need the last one
        if 'Adjustments' in name:
            logging.debug(name)

            try:
                conn = sqlite3.connect(db_path)
                df = pd.read_csv(os.path.join(root, name),
#                                dtype={ContractId               int64
#                                        CustomerId               int64
#                                        CurrentAccountNumber     int64
#                                        BalanceChangeType       object
#                                        CreatedAt               object
#                                        Amount                   int64}
                                ) #parse_dates=['RegistrationDate'],)
                df.to_sql('Adjustments', con=conn, if_exists='append', index=False)
                conn.commit()
                conn.close()
                logging.info('{} committed successfully'.format(name))
                del df

            except Exception as e:
                logging.exception('read_csv failed: {} \n {}'.format(name, e))
        # end for
logging.info('Completed successfully')


/jet/var/python/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [6]:
last_contract_file_path = '/mnt/disks/disk-1/d_light/Contracts/Contracts_2020_11_26.csv'
conn = sqlite3.connect(db_path)
df = pd.read_csv(last_contract_file_path)
df.to_sql('Contracts', con=conn, if_exists='append', index=False)
conn.commit()
conn.close()
                